Load some utils function + annotations first

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from utils import load_image_annotations, display_image_with_keypoints

In [ ]:
annotations = json.load(open("/root/data/gtsf_2.0/registration_test/gtsf_in_air_labels.json"))

In this example, we are going to load ~10 images as reference and use the mean of prediction to get the keypoints

In [ ]:
import numpy as np

In [ ]:
reference_masks = []
reference_kp = []
reference_images = []
reference_names = []
for ann in annotations:
    masked_image, static_mask, keypoints = load_image_annotations(ann, new_shape=(512, 512))
    reference_images.append(masked_image)
    reference_masks.append(static_mask)
    reference_kp.append(keypoints)
    reference_names.append(ann["External ID"])

In [ ]:
# import matplotlib.pyplot as plt

# plt.imshow(static_mask)
# for (kp, v) in keypoints.items():
#     _map = v["map"]
#     coord = np.where(_map > 0)
#     plt.scatter(coord[0], coord[1])
# plt.show()

In [ ]:
# plt.imshow(static_mask)
# for (kp, v) in keypoints.items():
#     coord = v["coord"]
#     print(coord)
#     plt.scatter(coord[0], coord[1])
# plt.show()

In [ ]:
print("Number of reference images: {}".format(len(reference_masks)))

Looping through all the examples.

In [ ]:
from copy import copy

from dipy.viz import regtools

from utils import translate_moving, register, display_warped_keypoints, display_pairs_with_keypoints, calculate_errors
from utils import display_keypoints_gt_pred
from utils import create_mask
import glob
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np

In [ ]:
display = False
# min_hsv = (50, 100, 100)
# max_hsv = (80, 255, 255)
min_hsv = (80, 20, 15)
max_hsv = (150, 260, 120)

In [ ]:
import random

In [ ]:
files = glob.glob("/root/data/phase_I/small-pen-test-site/1/2019-03-18/*/raw/*")
# random.shuffle(files)

In [ ]:
from skimage.measure import label

In [ ]:
all_images = []
for f in files:
    print(f)
    # first let's create the mask
    image, mask = create_mask(f, min_hsv, max_hsv)
    
    # flip cause reference are facing right
    image = np.fliplr(image)
    mask = np.fliplr(mask)
    
    moving_mask = np.array(mask, dtype=np.uint8)
    
#     mask = cv2.dilate(mask, np.ones((5, 5)))
#     moving_mask = cv2.erode(mask, np.ones((5, 5)))    
    
#     # we assume perfect mask 
#     m = cv2.imread("/root/data/gtsf_2.0/mask.png")
#     m[m>0] = 1
#     m = m[..., 0]
#     moving_mask = cv2.resize(m, (512, 512))
    
#     print(f)
#     plt.figure(figsize=(5, 5))
#     plt.imshow(image*np.expand_dims(moving_mask, axis=2))
#     plt.show()
#     continue
    
    total_kp = np.zeros_like(mask)
    kpdic = {}
    kpdic["image_path"] = f
    
    # loop through the reference map
    for idx in range(len(reference_masks)):
        kpdic["reference_name"] = reference_names[idx]
        static_mask = reference_masks[idx]
#         regtools.overlay_images(static_mask, 
#                                 moving_mask, 
#                                 'Static', 'Overlay', 'Moving', 'input_images.png')
        
        moving_translated, translation_vector = translate_moving(static_mask, moving_mask)
        
#         regtools.overlay_images(reference_masks[i], 
#                                 moving_translated, 
#                                 'Static', 'Overlay', 'Moving', 'input_images.png')

        # register
        mapping = register(static_mask, moving_translated)
#         plt.imshow(mapping.transform_inverse(static_mask))
#         plt.show()
        
        # warped keypoints
#         plt.figure(figsize=(15, 15))
#         plt.imshow(image*np.expand_dims(moving_mask, axis=2))
        kpdic["keypoints"] = {}
        for (k, v) in reference_kp[idx].items():
            
            kpmap = v["map"]
            warped_kp_map = mapping.transform_inverse(kpmap, "linear")        
            x, y = np.where(warped_kp_map > 0)
            kp_coord = [int(np.mean(x)) - translation_vector[0], 
                        int(np.mean(y)) - translation_vector[1]]
            kpdic["keypoints"][k] = kp_coord
#             plt.scatter(kp_coord[1], kp_coord[0], c="r") 
#         plt.show()            
        

        # display_warped_keypoints(image, warped_kp_map, translation_vector)
    # display_warped_keypoints(image, total_kp, [0, 0])
    with open(f.replace("jpg", "json").replace("raw","") , "w") as f:
        json.dump(kpdic, f)

In [ ]:
files[0]

In [ ]:
translation_vector

In [ ]:
translation_vector

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(reference_images[0])
for (k, v) in reference_kp[0].items():
    coord = v["coord"]
    #plt.scatter(coord[0], coord[1])
    x,y = np.where(v["map"]>0)
    plt.scatter(x,y)
    
plt.show()

In [ ]:
plt.imshow(mapping.transform_inverse(static_mask))